In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
anime = pd.read_csv("../input/anime-recommendations-database/anime.csv")

In [ ]:
anime.head(5)
anime.shape

In [ ]:
rating = pd.read_csv("../input/anime-recommendations-database/rating.csv")
rating.shape

In [ ]:
rating.tail(5)

In [ ]:
anime.isnull().sum()

In [ ]:
anime.fillna({'rating':0},inplace= True)

In [ ]:
anime['rating'].isnull().sum()

In [ ]:
anime_rating_count=rating.groupby(by='anime_id').count()['rating'].reset_index().rename(columns={'rating':'rating_count'})

In [ ]:
anime_rating_count.describe()

In [ ]:
filtered_anime=anime_rating_count[anime_rating_count['rating_count']>250]

In [ ]:
filtered_anime

In [ ]:
filtered_anime['rating_count']<250

In [ ]:
user_rating_count=rating.groupby(by='user_id').count()['rating'].reset_index().rename(columns={'rating':'user_rating'})
user_rating_count.describe()

In [ ]:
filtered_users=user_rating_count[user_rating_count['user_rating']>100]

In [ ]:
filtered_users['user_rating']>100


In [ ]:
filtered_rating_anime=rating[rating['anime_id'].isin(filtered_anime['anime_id'])]
filtered_rating=filtered_rating_anime[filtered_rating_anime['user_id'].isin(filtered_users['user_id'])]

In [ ]:
rating_matrix=filtered_rating.pivot_table(index='anime_id',columns='user_id',values='rating').fillna(0)

In [ ]:
from scipy.sparse import csr_matrix
csr_rating_matrix=csr_matrix(rating_matrix.values)
print(csr_rating_matrix)

In [ ]:
from sklearn.neighbors import NearestNeighbors
recommender=NearestNeighbors(metric='cosine')
recommender.fit(csr_rating_matrix)

In [ ]:
def recommendation_model(x):
    user_anime=anime[anime['name']==x]
    
    user_anime_index=np.where(rating_matrix.index==int(user_anime['anime_id']))[0][0]
    user_anime_ratings=rating_matrix.iloc[user_anime_index]
    user_anime_ratings_reshaped=user_anime_ratings.values.reshape(1,-1)
    distances, indices=recommender.kneighbors(user_anime_ratings_reshaped,n_neighbors=11)
    nearest_neighbor_indices=rating_matrix.iloc[indices[0]].index[1:]
    nearest_neighbors=pd.DataFrame({'anime_id':nearest_neighbor_indices})
    output=pd.merge(nearest_neighbors,anime,on='anime_id',how='left')
    return output
    

In [ ]:
recommendation_model('Shingeki no Kyojin')